# 02 - Data PreparationClean data, handle missing values, create customer mapping.

## Load Extracted Data

In [ ]:
import pandas as pdimport numpy as npfrom pathlib import Pathimport warningswarnings.filterwarnings('ignore')BASE_PATH = Path('..')FEATURES_PATH = BASE_PATH / 'features_v2'# Load raw extraction or existing clean dataif (FEATURES_PATH / 'raw_extraction.csv').exists():    df = pd.read_csv(FEATURES_PATH / 'raw_extraction.csv')    print(f"Loaded raw_extraction.csv: {len(df):,} rows")elif (FEATURES_PATH / 'v2_fact_lineitem.csv').exists():    df = pd.read_csv(FEATURES_PATH / 'v2_fact_lineitem.csv')    print(f"Loaded v2_fact_lineitem.csv: {len(df):,} rows")else:    print("No data file found. Run 01_data_extraction.ipynb first.")    df.head()

## Data Quality Check

In [ ]:
print("=== Data Quality Report ===")print(f"\nTotal rows: {len(df):,}")print(f"\nNull values:")print(df.isnull().sum())print(f"\nData types:")print(df.dtypes)print(f"\nNumeric columns stats:")numeric_cols = df.select_dtypes(include=[np.number]).columnsprint(df[numeric_cols].describe())

## Clean SKU Codes

In [ ]:
# Clean SKU codesdf['sku'] = df['sku'].astype(str).str.strip()df['sku'] = df['sku'].str.replace(r'[^a-zA-Z0-9]', '', regex=True)# Remove invalid SKUsdf = df[df['sku'].str.len() > 0]df = df[~df['sku'].str.lower().isin(['nan', 'none', ''])]print(f"After SKU cleaning: {len(df):,} rows")print(f"Unique SKUs: {df['sku'].nunique():,}")

## Handle Missing Prices

In [ ]:
# Convert to numericdf['quantity'] = pd.to_numeric(df['quantity'], errors='coerce').fillna(0)df['unit_price'] = pd.to_numeric(df['unit_price'], errors='coerce').fillna(0)df['line_total'] = pd.to_numeric(df['line_total'], errors='coerce').fillna(0)# Infer missing prices from same-SKU transactionsmissing_price_mask = (df['unit_price'] == 0) & (df['line_total'] == 0)print(f"Rows with missing price: {missing_price_mask.sum():,}")# Get median price per SKUsku_prices = df[df['unit_price'] > 0].groupby('sku')['unit_price'].median()# Fill missing pricesdf.loc[missing_price_mask, 'unit_price'] = df.loc[missing_price_mask, 'sku'].map(sku_prices)# Recalculate line_total where missingdf.loc[df['line_total'] == 0, 'line_total'] = df['quantity'] * df['unit_price']print(f"After price inference:")print(f"  Still missing price: {(df['unit_price'] == 0).sum():,}")print(f"  Still missing total: {(df['line_total'] == 0).sum():,}")

## Customer ID Mapping

In [ ]:
# Check if customer_id existsif 'customer_id' in df.columns:    print("Customer ID column exists")    print(f"Unique customer_ids: {df['customer_id'].nunique()}")        # Check for mixed types (numeric vs string like HB_*)    sample_ids = df['customer_id'].dropna().astype(str).head(20).tolist()    print(f"Sample IDs: {sample_ids}")        # Create master customer ID mapping    customer_mapping = df[['customer_id', 'customer_name']].drop_duplicates()    customer_mapping = customer_mapping.dropna(subset=['customer_id'])        # Deduplicate by customer_name (keep first)    customer_mapping = customer_mapping.drop_duplicates(subset=['customer_name'], keep='first')        # Assign sequential integer IDs    customer_mapping['master_customer_id'] = range(1, len(customer_mapping) + 1)        print(f"\nCustomer mapping created: {len(customer_mapping)} unique customers")    customer_mapping.head(10)else:    print("No customer_id column found")

## Save Cleaned Data

In [ ]:
# Save cleaned fact tabledf.to_csv(FEATURES_PATH / 'transactions_clean.csv', index=False)print(f"✓ Saved transactions_clean.csv: {len(df):,} rows")# Save customer mapping if createdif 'customer_mapping' in dir():    customer_mapping.to_csv(FEATURES_PATH / 'customer_id_mapping.csv', index=False)    print(f"✓ Saved customer_id_mapping.csv: {len(customer_mapping)} customers")